In [213]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome('./chromedriver.exe')
driver.get('https://www.slowand.com/')
time.sleep(3)

C:\Users\sinjy\AppData\Local\Temp\ipykernel_22516\1483214788.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')


In [214]:
cloth_categories = {'TOP', 'SHIRT', 'KNIT', 'BOTTOM', 'OUTER', 'OPS/SK'}

In [215]:
a_categories = []
for a in driver.find_elements(By.CSS_SELECTOR, "a"):
    if a.text in cloth_categories:
        a_categories += [a]
a_categories

[<selenium.webdriver.remote.webelement.WebElement (session="db9f0997edd516ffc71a9a87dc65d508", element="22CAE345B13E8405E6904D9FE55021B7_element_62")>,
 <selenium.webdriver.remote.webelement.WebElement (session="db9f0997edd516ffc71a9a87dc65d508", element="22CAE345B13E8405E6904D9FE55021B7_element_63")>,
 <selenium.webdriver.remote.webelement.WebElement (session="db9f0997edd516ffc71a9a87dc65d508", element="22CAE345B13E8405E6904D9FE55021B7_element_64")>,
 <selenium.webdriver.remote.webelement.WebElement (session="db9f0997edd516ffc71a9a87dc65d508", element="22CAE345B13E8405E6904D9FE55021B7_element_65")>,
 <selenium.webdriver.remote.webelement.WebElement (session="db9f0997edd516ffc71a9a87dc65d508", element="22CAE345B13E8405E6904D9FE55021B7_element_66")>,
 <selenium.webdriver.remote.webelement.WebElement (session="db9f0997edd516ffc71a9a87dc65d508", element="22CAE345B13E8405E6904D9FE55021B7_element_67")>]

In [216]:
a_cat = a_categories[0]
a_cat.click()

In [217]:
detail_cats = driver.find_element(By.CSS_SELECTOR, 'ul.menuCategory').find_elements(By.CSS_SELECTOR, 'a')

In [218]:
detail_cat = detail_cats[0]
detail_cat.click()

In [220]:
driver.find_elements(By.XPATH, '//*[@id="contents"]/main/div[3]/ol/li/a')[1].click()

In [195]:
a_products = driver.find_elements(By.XPATH, '//*[@id="contents"]/main/div[2]/div/ul/li/div[1]/div[2]/a')

In [198]:
a_products[0].send_keys(Keys.ENTER)

In [210]:
driver.switch_to.frame('crema-fit-product-combined-detail-1')

In [211]:
driver.find_element(By.XPATH, '//*[@id="fit-product-size-section"]/div[2]/div/table/tbody/tr/td[6]/span').text

'55'

### crawling func

In [39]:
from collections import defaultdict
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

def get_spec(url):
    driver = webdriver.Chrome('./chromedriver.exe')
    driver.get(url)
    time.sleep(3)

    cloth_categories = {'TOP', 'SHIRT', 'KNIT', 'BOTTOM', 'OUTER', 'OPS/SK'}
    cloth_dict = defaultdict(dict)

    detail_cats_len = len(driver.find_elements(By.XPATH, '//*[@id="category"]/div/ul/li/a'))
    for m in range(detail_cats_len):
        a_cat = driver.find_elements(By.XPATH, '//*[@id="category"]/div/ul/li/a')[m]
        cat_tmp = a_cat.text
        if not (cat_tmp in cloth_categories):
            continue
        a_cat.send_keys(Keys.ENTER) 
        time.sleep(3)

        # detail_cats_len = len(driver.find_element(By.CSS_SELECTOR, 'ul.menuCategory').find_elements(By.CSS_SELECTOR, 'a'))
        detail_cats_len = len(driver.find_elements(By.XPATH, '//*[@id="contents"]/main/div[1]/div[3]/div[1]/ul/li/a'))
        for k in range(detail_cats_len):
            detail_cat = driver.find_elements(By.XPATH, '//*[@id="contents"]/main/div[1]/div[3]/div[1]/ul/li/a')[k]
            detail_cat_tmp = detail_cat.text
            cloth_dict[cat_tmp][detail_cat_tmp] = []
            detail_cat.send_keys(Keys.ENTER)
            time.sleep(3)

            detail_pages_len = len(driver.find_elements(By.XPATH, '//*[@id="contents"]/main/div[3]/ol/li/a'))
            # detail_pages_len = 5
            for j in range(detail_pages_len):
                a_detail_page = driver.find_elements(By.XPATH, '//*[@id="contents"]/main/div[3]/ol/li/a')[j]
                a_detail_page.send_keys(Keys.ENTER)
                time.sleep(3)

                products_len = len(driver.find_elements(By.XPATH, '//*[@id="contents"]/main/div[2]/div/ul/li/div[1]/div[2]/a'))
                # products_len = 5
                for i in range(products_len):
                    a_product = driver.find_elements(By.XPATH, '//*[@id="contents"]/main/div[2]/div/ul/li/div[1]/div[2]/a')[i]
                    a_product.send_keys(Keys.ENTER)
                    time.sleep(5)
                    try:
                        driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="crema-fit-product-combined-detail-1"]'))
                    except:
                        time.sleep(20)
                        driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="crema-fit-product-combined-detail-1"]'))
                    try:
                        spec = float(driver.find_elements(By.XPATH, '//*[@id="fit-product-size-section"]/div[2]/div/table/tbody/tr/td/span')[-1].get_attribute('innerText'))
                        print(spec)
                        cloth_dict[cat_tmp][detail_cat_tmp] += [spec]
                    
                    except:
                        print("data miss")
                        pass
                    driver.back()
                    time.sleep(3)
    return cloth_dict
                

In [40]:
import pickle

cloth_dict = get_spec('https://www.slowand.com/')
with open('cloth_dict.pickle', 'wb') as f:
    pickle.dump(cloth_dict, f)

C:\Users\sinjy\AppData\Local\Temp\ipykernel_14716\1726580353.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="crema-fit-product-combined-detail-1"]"}
  (Session info: chrome=113.0.5672.93)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00AD8893+48451]
	(No symbol) [0x00A6B8A1]
	(No symbol) [0x00975058]
	(No symbol) [0x009A0467]
	(No symbol) [0x009A069B]
	(No symbol) [0x009CDD92]
	(No symbol) [0x009BA304]
	(No symbol) [0x009CC482]
	(No symbol) [0x009BA0B6]
	(No symbol) [0x00997E08]
	(No symbol) [0x00998F2D]
	GetHandleVerifier [0x00D38E3A+2540266]
	GetHandleVerifier [0x00D78959+2801161]
	GetHandleVerifier [0x00D7295C+2776588]
	GetHandleVerifier [0x00B62280+612144]
	(No symbol) [0x00A74F6C]
	(No symbol) [0x00A711D8]
	(No symbol) [0x00A712BB]
	(No symbol) [0x00A64857]
	BaseThreadInitThunk [0x774A00C9+25]
	RtlGetAppContainerNamedObjectPath [0x77BD7B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77BD7B1E+238]


In [42]:
driver = webdriver.Chrome('./chromedriver.exe')
driver.get('https://www.slowand.com/')
time.sleep(3)

cloth_categories = {'TOP', 'SHIRT', 'KNIT', 'BOTTOM', 'OUTER', 'OPS/SK'}
cloth_dict = defaultdict(dict)

detail_cats_len = len(driver.find_elements(By.XPATH, '//*[@id="category"]/div/ul/li/a'))
for m in range(detail_cats_len):
    a_cat = driver.find_elements(By.XPATH, '//*[@id="category"]/div/ul/li/a')[m]
    cat_tmp = a_cat.text
    if not (cat_tmp in cloth_categories):
        continue
    a_cat.send_keys(Keys.ENTER) 
    time.sleep(3)

    # detail_cats_len = len(driver.find_element(By.CSS_SELECTOR, 'ul.menuCategory').find_elements(By.CSS_SELECTOR, 'a'))
    detail_cats_len = len(driver.find_elements(By.XPATH, '//*[@id="contents"]/main/div[1]/div[3]/div[1]/ul/li/a'))
    for k in range(detail_cats_len):
        detail_cat = driver.find_elements(By.XPATH, '//*[@id="contents"]/main/div[1]/div[3]/div[1]/ul/li/a')[k]
        detail_cat_tmp = detail_cat.text
        cloth_dict[cat_tmp][detail_cat_tmp] = []
        detail_cat.send_keys(Keys.ENTER)
        time.sleep(3)

        detail_pages_len = len(driver.find_elements(By.XPATH, '//*[@id="contents"]/main/div[3]/ol/li/a'))
        # detail_pages_len = 5
        for j in range(detail_pages_len):
            a_detail_page = driver.find_elements(By.XPATH, '//*[@id="contents"]/main/div[3]/ol/li/a')[j]
            a_detail_page.send_keys(Keys.ENTER)
            time.sleep(3)

            products_len = len(driver.find_elements(By.XPATH, '//*[@id="contents"]/main/div[2]/div/ul/li/div[1]/div[2]/a'))
            # products_len = 5
            for i in range(products_len):
                a_product = driver.find_elements(By.XPATH, '//*[@id="contents"]/main/div[2]/div/ul/li/div[1]/div[2]/a')[i]
                a_product.send_keys(Keys.ENTER)
                time.sleep(5)
                try:
                    driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="crema-fit-product-combined-detail-1"]'))
                except:
                    time.sleep(20)
                    driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="crema-fit-product-combined-detail-1"]'))
                try:
                    spec = float(driver.find_elements(By.XPATH, '//*[@id="fit-product-size-section"]/div[2]/div/table/tbody/tr/td/span')[-1].get_attribute('innerText'))
                    print(spec)
                    cloth_dict[cat_tmp][detail_cat_tmp] += [spec]
                
                except:
                    print("data miss")
                    pass
                driver.back()
                time.sleep(3)

C:\Users\sinjy\AppData\Local\Temp\ipykernel_14716\3501387367.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')


61.5
60.0
52.0
55.0
55.0
62.0
data miss
69.0
59.5
54.0
68.5
58.0
62.0
61.0
64.5
42.5
43.0
55.0
48.0
68.0
58.0
51.0
40.0
72.0
55.5
60.0
65.0
data miss
data miss
70.0
49.0
54.0
54.0
57.5
54.0
70.0
56.0
55.0
64.5
64.0
47.5
49.0
53.0
57.5
data miss
62.0
50.0
48.0
50.0
53.0
59.5
64.0
57.0
64.0
53.0
47.5
43.0
51.5
56.0
56.0
63.0
60.0
59.0
58.0
56.0
55.0
63.5
58.5
57.5
66.0
58.0
data miss
51.0
66.0
55.0
55.0
49.0
62.0
40.0
66.0
70.5
64.0
65.0
52.5
52.0
47.5
55.0
64.0
64.0
56.5
51.0
25.0
53.0
58.0
44.5
46.5
39.0
45.5
52.0
51.0
39.0
47.0
44.0
49.0
45.0
50.0
49.0
42.5
27.0
41.0
38.0
46.5
51.5
53.0
50.0
data miss
67.0
data miss
68.0
data miss
data miss
65.0
data miss
data miss
data miss
data miss
data miss
data miss
data miss
data miss
67.0
data miss
64.5
data miss
66.0
67.0
data miss
data miss
67.0
70.0
data miss
63.5
75.5
72.0
72.0
data miss
58.0
59.0
52.0
data miss
71.5
68.0
data miss
53.5
data miss
data miss
data miss
data miss
55.0
45.5
48.0
61.0
48.0
57.0
66.0
data miss
47.5
67.5
53.0
46.0


KeyboardInterrupt: 

In [44]:
import pickle
with open('cloth_dict', 'wb') as f:
    pickle.dump(cloth_dict, f)
print("success")

success
